In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_json("/content/drive/MyDrive/NLP/Project/datasets.json")

In [ ]:
import random
def make_trait(row):
    row["trait_0"] = 0.0 if row["mbti_result"][0] == "I" else 1.0
    row["trait_1"] = 0.0 if row["mbti_result"][1] == "N" else 1.0
    row["trait_2"] = 0.0 if row["mbti_result"][2] == "T" else 1.0
    row["trait_3"] = 0.0 if row["mbti_result"][3] == "J" else 1.0
    tweets_selected = random.sample(row["tweets"], 10)
    row["tweets_text"] = " ".join(tweets_selected)
    row["tweets"] = tweets_selected
    return row
df = df.apply(make_trait, axis=1)

# Fine Tuning

In [ ]:
import pandas as pd
import openai
import subprocess


df_fine_tune = df.loc[:,["mbti_result", "tweets_text"]]
df_fine_tune.rename(columns={'mbti_result':'prompt', 'tweets_text':'completion'}, inplace=True)
df_fine_tune.to_csv('prepared_data.csv',index=False)


## prepared_data.csv --> prepared_data_prepared.json
subprocess.run('openai tools fine_tunes.prepare_data --file prepared_data.csv --quiet'.split())

## Start fine-tuning
subprocess.run('openai api fine_tunes.create --training_file prepared_data_prepared.jsonl --model text-davinci-003 --suffix "PersonalityModel"'.split())

# Data Augmentation

In [ ]:
labels = list(df["mbti_result"].unique())

In [ ]:
len(labels)

16

In [ ]:
df.sample(n=3)

,twitter_id,mbti_result,tweets,trait_0,trait_1,trait_2,trait_3,tweets_text
2216,matinkh_fa,INTJ,"[تو چه شرایطی فکر می کنی اعتماد از بین میره, و...",0.0,0.0,0.0,0.0,تو چه شرایطی فکر می کنی اعتماد از بین میره واق...
230,deldadew,INTP,[نگفتیا چقدر میشه از توییتر بیایم تخفیفم داریم...,0.0,0.0,0.0,1.0,نگفتیا چقدر میشه از توییتر بیایم تخفیفم داریم ...
1052,MissFatha77,ESFJ,[به نظرم بعضیاشون خودشون نمی دونن و شاید حس کن...,1.0,1.0,1.0,0.0,به نظرم بعضیاشون خودشون نمی دونن و شاید حس کنن...


In [ ]:
augmented_text_on_labels = {}
for label in labels:
  augmented_text_on_labels[label] = []

In [ ]:
import openai

openai.api_key = 'has been cleared for secrecy reason :)'

grouped_by_labels = df.groupby("mbti_result")

for label, frame in grouped_by_labels:
  print(f"for label : {label}")
  for _ in range(5):
    tmp_frame = frame.sample(n=5)
    tweets = tmp_frame["tweets"]
    # tweets = tweets.iloc
    prompt = "I want you to act as a Data Augmenter Specialist. I will provide you some users tweets and their related MBTI types."
    prompt += " And then I want you to make a tweet that a person with that MBTI type would tweet. tweets examples are provided below:\n"
    for tweet in tweets:
        tmp_d = {"tweet" : " ".join(tweet[:5]), "mbti_type" : label}
        prompt += str(tmp_d)
        prompt += "\n"

    chat = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      temperature=0.3,
      max_tokens=1200,
      n=1,
      stop=None
    )
    reply = chat.choices[0].text.strip().strip("\n").replace(label, "").replace("tweet", "").replace(":", "").replace("Output", "").replace("output", "").replace("Tweet", "").replace("Answer", "")
    augmented_text_on_labels[label].append(reply)
    print("chatgpt : ", reply)
    # break


for label : ENFJ
chatgpt :    با کسانی که باهاشون راحتی حرف میزنی، به طبیعت برو و با انواع موزیک ها خودتو راهنمایی کن تا بتونی به خاطرات خوبت بپردازی و به خودت بگویی که هیچ چیز نمیتونه تو رو از راه رفتنت بگیره!
chatgpt :    امروز به دنبال یک روز خوب و محبوب هستم که بتونم با دوستانم به گرمی به یاد باشه و احساساتمونو با هم به اشتراک بذاریم! #دوست_داشتنی #خوش_اندیشی
chatgpt :    امروز به یک تجربه ی خاص و دیدنی بروزکردم که با دوستانم به خوبی به آن بپردازیم و با انتقادات و پیشنهاداتی که به آن اضافه کردیم، به تصمیم گیری یک نوع جدید از زندگی را برای همه تجربه کردیم! #زندگی_خوب
chatgpt :   for  امروز با دوستانم که به هم دوستی داریم کاملا به آرامش و خوشحالی فراموش نشده! قلبم با شادی خاطره ای به آن ها بسته است. #HappyMoments #FriendsForLife
chatgpt :  Answer امروز با دیدن کسی که با ایده های خودش به خودش رحم میکنه، خیلی خوشحال شدم. امیدوارم که همه ی افراد بتونن با خودشون به خودشون رحم باشن و از خودشون بیشتر بخوانن. #رحمت #خودشون #ایده
for label : ENFP
chatgpt :    به خاطر پیشرفت فناوری، امروزه به

In [ ]:
for key, value in augmented_text_on_labels.items():
  for tweet in value:
    new_row = pd.DataFrame({"tweets_text" : [tweet], "mbti_result" : [key]})
    df = pd.concat([df, new_row], axis=0 ,ignore_index=True)

In [ ]:
df.tail(20)

,twitter_id,mbti_result,tweets,trait_0,trait_1,trait_2,trait_3,tweets_text
4016,NaN,ESFP,NaN,NaN,NaN,NaN,NaN,Answer امروز روزی خوبی بوده، که به نظرم باید ب...
4017,NaN,ESFP,NaN,NaN,NaN,NaN,NaN,Answer امروز چیز خوبی برای کار کردن نداشتم، بل...
4018,NaN,ESFP,NaN,NaN,NaN,NaN,NaN,امروز به بازی های بازیکنانی که با تو بازی می...
4019,NaN,ESFP,NaN,NaN,NaN,NaN,NaN,امروز برای اولین بار به باشگاه فوتبال رفتم! ...
4020,NaN,ESFP,NaN,NaN,NaN,NaN,NaN,امروز به همه نگاه کردم و دیدم که چقدر همه خو...
4021,NaN,ESTP,NaN,NaN,NaN,NaN,NaN,Answer به نظر میرسه که همه ی چیزا به خوبی رفته...
4022,NaN,ESTP,NaN,NaN,NaN,NaN,NaN,Answer امروز با دوستام برنامه ریزی کردیم که چی...
4023,NaN,ESTP,NaN,NaN,NaN,NaN,NaN,برای انجام کاری باید به سرعت به کار برویم، ن...
4024,NaN,ESTP,NaN,NaN,NaN,NaN,NaN,Answer اونا که امتحاناشون سخت نکنند کاری ندارن...
4025,NaN,ESTP,NaN,NaN,NaN,NaN,NaN,Answer امروز به همه چیز بروز شدم! از کارت به پ...


In [ ]:
df.to_json("/content/drive/MyDrive/NLP/Project/datasets_augmented.json")